# Extract Training Features


## Background

**Training data** is the most important part of any supervised machine learning workflow. The quality of the training data has a greater impact on the classification than the algorithm used. Large and accurate training data sets are preferable: increasing the training sample size results in increased classification accuracy ([Maxell et al 2018](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343)).  A review of training data methods in the context of Earth Observation is available [here](https://www.mdpi.com/2072-4292/12/6/1034) 

There are many platforms to use for gathering land cover training labels, the best one to use depends on your application. GIS platforms are great for collection training data as they are highly flexible and mature platforms; [Geo-Wiki](https://www.geo-wiki.org/) and [Collect Earth Online](https://collect.earth/home) are two open-source websites that may also be useful depending on the reference data strategy employed. Alternatively, there are many pre-existing training datasets on the web that may be useful, e.g. [Radiant Earth](https://www.radiant.earth/) manages a growing number of reference datasets for use by anyone. With locations of land cover labels available, we can extract features at these locations from satellite imagery as input for machine learning.

## Description
This notebook will extract training feature layers from the `open-data-cube` of Sentinel-2 multispectral images using geometries within a geojson.

To do this, we rely on a custom function called `collect_training_data`, contained within the [deafrica_tools.classification](../../../../Tools/deafrica_tools/classification.py) script.  The principal goal of this notebook is to familarise users with this function so they can extract the appropriate data for their use-case. The default example also highlights extracting a set of useful feature layers for land cover classification. The workflow includes the following steps:

1. Preview the points in our training data by plotting them on a basemap
2. Define a feature layer function to pass to `collect_training_data`
3. Extract training features from the datacube using `collect_training_data`
4. Export the training data to disk for use in subsequent scripts

***

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages


In [ ]:
%matplotlib inline

import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
from odc.io.cgroups import get_cpu_quota
from odc.algo import xr_geomedian
from deafrica_tools.plotting import map_shapefile
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import collect_training_data

## Analysis parameters

* `training_data_path`: The path to the input vector file from which we will extract training data. A default geojson is provided for this notebook.
* `class_attr`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**
>**Note**: If you wish to run your own classification workflow, the first step is to replace this training data with your own in the '/Data' folder. The training data can be in other formats (e.g. shapefile) that can be read by `geopandas` but need to have a corresponding class label attribute field `class_attr`.
* `output_crs`: Output spatial reference system.

In [ ]:
training_data_path = 'Data/train_poly_848_20171124_class_merged_buffered_smaller.geojson'
class_attr = 'LC_Class_I' # class label in integer format
output_crs='epsg:32736' # WGS84/UTM Zone 36S

## Load input data

We can load and preview our input data shapefile using `geopandas`. The shapefile should contain a column with class labels (e.g. 'class_attr') which must be integer type. These labels will be used to train our model. 

In [ ]:
# Load input data shapefile
training_data= gpd.read_file(training_data_path) # read training points as geopandas dataframe
training_data=training_data[[class_attr,'geometry']] # select attributes
# Plot first five rows
training_data.head()

Now let's plot some training data in an interactive map:

In [ ]:
# 1:'Forest',5:'Grassland',7:'Shrubland',9:'Perennial Cropland',10:'Annual Cropland',11:'Wetland',12:'Water Body',13:'Urban Settlement'
# Satellite hybrid: https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}
# Satellite: https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}
# map_shapefile(training_data[0:10], attribute=class_attr,continuous=False, hover_col=True)
training_data_subset = training_data[0:10]

training_data_subset.explore(
    tiles = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}", 
    attr ='Imagery @2022 Landsat/Copernicus, Map data @2022 Google',
    popup=True,
    cmap='viridis',
    style_kwds=dict(radius= 5, color= 'red', fillOpacity= 0.8, fillColor= 'red', weight= 3),
    )

This interactive map plots the first ten records from our input vector file.

## Defining Query

The function `collect_training_data` takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any `NaN` or `inf` values.

The below variables can be set within the `collect_training_data` function:

* `feild`: The name of column in your geojson file attribute table that contains the class labels, which corresponds to the `class_attr` that we defined earlier.
* `zonal_stats`: An optional string giving the names of zonal statistics to calculate across each geometry (polygon or point). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.
* `dc_query`: A datacube query dictionary for the Open Data Cube query such as `measurements` (the bands to load from the satellite), the `resolution` (the cell size), and the `output_crs` (the output projection). 
* `feature_func`:  A function for generating feature layers that is applied to the data within the bounds of the input geometry. This function will take the 'dc_query' as the only argument.
* `return_coords`: If True, then the training data will contain two extra columns ‘x_coord’ and ‘y_coord’ corresponding to the x,y coordinate of each sample.

> Note: `collect_training_data` also has a number of additional parameters for handling ODC I/O read failures, where polygons that return an excessive number of null values can be resubmitted to the multiprocessing queue.  Check out the [docs](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/83116e80ebb4f8744e3de74e7a713aadd0a7577a/Tools/deafrica_tools/classification.py#L565) to learn more.

we will define the first three parameters and describe the `feature_func` seperately in a moment.

In [ ]:
#set up our inputs to collect_training_data
zonal_stats = None
# Set up the inputs for the ODC query
time = ('2021')
# using nine spectral bands with 10~20 m spatial resolution
measurements = ['blue','green','red','red_edge_1','red_edge_2', 'red_edge_3','nir_1','swir_1','swir_2']
resolution = (-10,10)

Note that we've selected nine spectral bands with spatial resolution no lower than 20 m here for demonstration. However, it is advised that you test and select the bands based on your own classification task. Using the variables above, we can generate a datacube query object from the parameters above:

In [ ]:
query = {
    'time': time,
    'measurements': measurements,
    'output_crs': output_crs,
    'resolution': resolution
}

## Defining feature function

To create the desired feature layers, we pass instructions to `collect_training_data` through the `feature_func` parameter. The `feature_func` must accept a `dc_query` dictionary, and return a single `xarray.Dataset` or `xarray.DataArray` containing 2D coordinates (i.e x, y - no time dimension). e.g.

          def feature_function(query):
              dc = datacube.Datacube(app='feature_layers')
              ds = dc.load(**query)
              ds = ds.mean('time')
              return ds

Below, we will define a more complicated feature layer function than the brief example shown above. Firstly We will calculate the Normalised Difference Vegetation Index (NDVI), which is commonly used to distinguish between vegetation and non-vegetation land cover classes. We use the `calculate_indices`function to automatically calculate NDVI for all specified bands. The `calculate_indicies` function provides an easier way to calculate a wide range of remote sensing indices, without explicitly needing to write the code of the band math. More information on calculating band indices can be found in [DE Africa’s documentation](https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Frequently_used_code/Calculating_band_indices.html). 


In addition, we'll use temporal signatures to help distinguish land cover classes with seasonal patterns (e.g. croplands) with others. To reduce data size while keeping seasonal changes, we are implementing bimonthly temporal aggregation, i.e. geomedian (sometimes referred to as the 'geometric median') for each pixel location. The significance of the geomedian is that all bands are considered simultaneously. This maintains the spectral relationship between bands, providing the most representative value. Additionally, the geomedian statistic reduces spatial noise and improves colour balance compared to similar statistics such as the median or medoid. More information on geomedian and available geomedian products can be found in [DE Africa's documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/GeoMAD_specs.html).
> Note: A shorter time interval could be chosen to obtain a finer-grained temporal signatures, but it would also result in larger data gaps in the satellite time-series due to presence of cloud and cloud shadow. If cloud presence in your study site is very frequent, you may need to choose a courser temporal aggregates, e.g. every three months.

As the output of the feature layer function should only contain coordinates with no time dimension, we need to convert the temporal features of each measurement to multiple individual features. For example, we will rename and convert the bimonthly temporal features for blue band to blue_0, blue_2, ..., blue_5.

Putting all above together, our final feature layer function looks like this:

In [ ]:
# define a function to feature layers
def feature_layers(query): 
    # connect to the datacube so we can access DE Africa data
    dc = datacube.Datacube(app='feature_layers')
    
    # load Sentinel-2 analysis ready data
    ds = load_ard(dc=dc,
                  products=['s2_l2a'],
                  group_by='solar_day',
                  verbose=False,
                  **query)
    
    # calculate NDVI
    ds = calculate_indices(ds,
                           index=['NDVI'],
                           drop=False,
                           satellite_mission='s2')
    
    # interpolate nodata using mean of previous and next observation
#     ds=ds.interpolate_na(dim='time',method='linear',use_coordinate=False,fill_value='extrapolate')
#     ds=ds.interpolate_na(dim='time',method='linear',use_coordinate=False)

    # calculate bi-monthly geomedian
    ds=ds.resample(time='2MS').map(xr_geomedian)
    
    # stack multi-temporal measurements and rename them
    n_time=ds.dims['time']
    list_measurements=list(ds.keys())
    list_stack_measures=[]
    for j in range(len(list_measurements)):
        for k in range(n_time):
            variable_name=list_measurements[j]+'_'+str(k)
            measure_single=ds[list_measurements[j]].isel(time=k).rename(variable_name)
            list_stack_measures.append(measure_single)
    ds_stacked=xr.merge(list_stack_measures,compat='override')
    return ds_stacked

Now let's run the `collect_training_data` function. This may take minutes to hours depending on your number of training points, number of measurements/bands set for the query and the calculation work in the feature function. Since we've used 10 measurements (9 spectral bands and 1 NDVI index) with 6 temporal geomedian for each band, it can be very time-consuming to finish the training features extraction. Therefore, here we are passing in `gdf=training_data[0:10]` to only run the code over the first 10 geometries as demonstration. Nevertheless, the extracted full training data file is provided in the 'Results/' folder, which will be used for next module of the workflow.

> **Note**:  With supervised classification, its common to have many, many labelled geometries in the training data. `collect_training_data` can parallelize across the geometries in order to speed up the extracting of training data. Setting `ncpus>1` will automatically trigger the parallelization. However, its best to set `ncpus=1` to begin with to assist with debugging before triggering the parallelization.

In [ ]:
# detect the number of CPUs
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

# collect training data
column_names, model_input = collect_training_data(
    gdf=training_data[0:10], # replace with gdf=training_data if you are extracting all the training data
    dc_query=query,
    ncpus=ncpus,
    field=class_attr,
    zonal_stats=zonal_stats,
    feature_func=feature_layers,
    return_coords=True)

The function returns a list (`column_names`) contains a list of the names of the `class_attr` and all the features we've computed. As we set the parameter `return_coords` above as True the list also contains the coordinates of the points:

In [ ]:
print(column_names)

The second object returned by the function is a numpy.array (`model_input`) and contains the data from our labelled geometries. The first item in the array is the class integer (e.g. in the default example 61. 'Bare soils', 11. 'Tree crops', etc.), the second set of items are the values for each feature layer we computed and the coordinates:

In [ ]:
print(np.array_str(model_input, precision=2, suppress_small=True))

## Export training features

Once we've collected the training features we require, we can write the data to disk. For example to export the extracted first 10 training features as a text file:

In [ ]:
# convert the data to geopandas dataframe
pd_training_features=pd.DataFrame(data=model_input,columns=column_names)
#set the name and location of the output file
output_file = "Results/Mozambique_training_features_demo.txt"
#Export files to disk
pd_training_features.to_csv(output_file, header=True, index=None, sep=' ')

We would also like to allow us to check the locations of training points after training data filtering implemented through next notebook. To do that, we reconstruct a geopandas dataframe using the coordinates returned by the `collect_training_data` function and export the features as a geojson file:

In [ ]:
# create geopandas dataframe
gpd_training_features=gpd.GeoDataFrame(pd_training_features, 
geometry=gpd.points_from_xy(model_input[:,-2], model_input[:,-1],crs=output_crs))
# save as geojson file
output_file='Results/Mozambique_training_features_demo.geojson'
gpd_training_features.to_file(output_file, driver="GeoJSON")

The extracted full training features file is provided as 'Results/Mozambique_training_features.geojson', which will allow us to import the data in the next module(s) of the workflow.